In [46]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import math

In [4]:
df=pd.read_csv('Bank customer data.csv')

In [5]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
# Drop some features that will not be much of connected to the customer stay or leave bank.
df = df.drop({'RowNumber', 'CustomerId', 'Surname','Geography', 'Tenure'}, axis=1)

In [8]:
df.head()

,CreditScore,Gender,Age,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,Female,42,0.00,1,1,1,101348.88,1
1,608,Female,41,83807.86,1,0,1,112542.58,0
2,502,Female,42,159660.80,3,1,0,113931.57,1
3,699,Female,39,0.00,2,0,0,93826.63,0
4,850,Female,43,125510.82,1,1,1,79084.10,0


In [11]:
df.isnull().sum()

CreditScore        0
Gender             0
Age                0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [12]:
# change gender to 0 for male and 1 for female
def convert_gender(Gender):
    if(Gender=='Female'):
        return 1
    else:
        return 0

In [13]:
df['Gender'] = df['Gender'].apply(convert_gender)

In [14]:
df.head()

,CreditScore,Gender,Age,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,1,42,0.00,1,1,1,101348.88,1
1,608,1,41,83807.86,1,0,1,112542.58,0
2,502,1,42,159660.80,3,1,0,113931.57,1
3,699,1,39,0.00,2,0,0,93826.63,0
4,850,1,43,125510.82,1,1,1,79084.10,0


In [24]:
X = df.drop('Exited', axis=1)
y=df['Exited']

In [ ]:
# scale the data to prevent some features dominate the others. For example here, estimate salary and balance.

In [27]:
from sklearn.preprocessing import StandardScaler

In [28]:
scaler = StandardScaler()

In [29]:
X = scaler.fit_transform(X)

In [30]:
from sklearn.model_selection import train_test_split

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [42]:
X_train

array([[ 0.36701255, -0.91241915, -0.66001848, ...,  0.64609167,
         0.97024255,  1.37378358],
       [-0.19171334, -0.91241915,  0.29351742, ...,  0.64609167,
         0.97024255,  1.66740729],
       [-0.94702796, -0.91241915, -1.42284721, ...,  0.64609167,
        -1.03067011, -0.24691025],
       ..., 
       [ 0.87400456,  1.09598752, -0.08789694, ..., -1.54776799,
        -1.03067011, -0.13685352],
       [ 0.17042381, -0.91241915,  0.38887101, ...,  0.64609167,
        -1.03067011, -0.04490188],
       [ 0.48082708, -0.91241915,  1.15169974, ...,  0.64609167,
         0.97024255, -0.80874677]])

In [48]:
# declare hte variables with some values
regressors = len(X_train[0])
init_m = 0
init_b = [0 for x in range(regressors)]
learning_rate = 0.0001
num_iteration = 500
N = len(X_train)
landa = 100



# training set
def minimize_m_b(X_train,y_train,init_m1 , init_b1, learning_rate, landa):
    minimize_m = init_m1
    minimize_b = [x for x in init_b1] 
    # without setting up the values for minimize_b in this way, the values for both will keep adding up
    minimized_b = [0 for x in range(regressors)]
    
    for i in range(N):
        sum_m_b = 0 # forgot to put this and cause the huge numbers for m and b
        
        for j in range(regressors):
            sum_m_b += init_b1[j] * X_train[i][j]
            
        minimize_m +=  (2*y_train[i]-1)/(math.exp(init_m1 + sum_m_b)+1)
        
        for k in range(regressors):
            minimize_b[k] += (((2*y_train[i]+1)*x_train[i][k]) / (math.exp(init_m1+sum_m_b)+1)) + (x_train[i][k]*landa/N)
        
        
    minimized_m = init_m1 - (learning_rate/N) * minimize_m

    for l in range(regressors):
        minimized_b[l] = init_b1[l] - (learning_rate/N) * minimize_b[l]
    return [minimized_m, minimized_b]


def run_minimize_m_b(X_train,y_train,init_m, init_b, learning_rate, num_iteration, landa):
    init_m1 = init_m
    init_b1 = [x for x in init_b]
    for i in range(num_iteration):
        init_m1,init_b1 = minimize_m_b(X_train,y_train, init_m1, init_b1, learning_rate, landa)
    return [init_m1, init_b1, learning_rate, num_iteration, landa] # the problem for the huge numbers are here, after switch to m1 and b1, the number get smaller



m, b, learning_rate, iterations, landa = run_minimize_m_b(X_train,y_train, init_m, init_b, learning_rate, num_iteration, landa)
print ("intercept: ",m, "Slope: ",b)       

KeyError: 0